In [ ]:
from transformers import PerceiverModel

perciever_model = "deepmind/multimodal-perceiver"
perceiver = PerceiverModel.from_pretrained(model_name)

In [ ]:
inputs = ["Hello"]
outputs = self.perceiver(
    inputs=inputs,
)